In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import random
from helper_files import pick_batch

def gaussian_noise_layer(input_layer, std):
    noise = tf.random_normal(shape=tf.shape(input_layer), mean=0.0, stddev=std, dtype=tf.float32) 
    return input_layer + noise

def early_stop(current, winner, win_step, step, sess):
    saver = tf.train.Saver()
    path = "./saved_model/"

    if current > winner:
        saver.save(sess, path, step) #save the model
        win_step = step
        return winner, win_step
    else:
        if (step - win_step) >3000: # if no new winner for 3000 steps 
            with tf.Session() as sess: # restore the parameters
                saver.restore(sess, save_path)
        return -1, win_step

def conv1D_layer(inpt, filter_size, init_weights, init_bias):
    conv1d= tf.layers.conv2d(inpt,
                                  filters = filter_size,
                                   kernel_size = [2, int(inpt.shape[2])],
                                   strides=[1, int(inpt.shape[2])],
                                   padding="same",
                                   activation =  tf.nn.relu,
                                   kernel_initializer = init_weights,
                                   bias_initializer = init_bias)

    mp_conv1 = tf.layers.max_pooling2d(inputs = conv1d,
                                      pool_size = [2,1],
                                      strides = 2,
                                      padding = "SAME")
    
    mp_conv1_shape = mp_conv1.shape # shape of the output of the first layer
    mp_conv1= tf.reshape(mp_conv1,[-1,int(mp_conv1_shape[1]),int(mp_conv1_shape[3]), 1])
    mp_conv1_do = tf.layers.dropout(mp_conv1, rate=0.3, training = True) # DROPOUT
    print(mp_conv1_do)
    return mp_conv1_do    


In [2]:
sess = tf.InteractiveSession()
np.random.seed(25)
random.seed(25)



data = np.load('./data.npy')

x = data[0] # specto (nr_of_tracks, 647, 128)
y = data[1] # label

# create train/test set
(x_train, x_val, y_train, y_val) = train_test_split(x, y, test_size=0.3)


##### declare varibales
input_shape = x_train.shape[1:3] #(647, 128)
input_data = tf.placeholder(tf.float32,shape=[None,input_shape[0], input_shape[1]])
input_layer = tf.reshape(input_data,[-1,input_shape[0],input_shape[1], 1]) #[batch_size, timesteps, frequencies, channels]
#input_rhsp = gaussian_noise_layer(input_rshp, 0.1) # Add noise

# define initilizers
init_weights = tf.variance_scaling_initializer(scale = 2.0, mode= 'fan_in', distribution= 'uniform')
init_bias= tf.constant_initializer(0.1)

fltrs=[64,128,256,512,1024,2048,4096, 4096, 4096,4096] #65-70ish%
#fltrs=[64,128,256,256,512,512,512,512,1024,1024, 1024, 2048,2048,2048,4096,4096] #65-70ish%
len(fltrs)

10

In [3]:
for idx, fltr in enumerate(fltrs):
    print("layer ",idx+1)
    input_layer = conv1D_layer(input_layer, fltr, init_weights, init_bias)
    print()

layer  1
Tensor("dropout/dropout/mul:0", shape=(?, 324, 64, 1), dtype=float32)

layer  2
Tensor("dropout_1/dropout/mul:0", shape=(?, 162, 128, 1), dtype=float32)

layer  3
Tensor("dropout_2/dropout/mul:0", shape=(?, 81, 256, 1), dtype=float32)

layer  4
Tensor("dropout_3/dropout/mul:0", shape=(?, 41, 512, 1), dtype=float32)

layer  5
Tensor("dropout_4/dropout/mul:0", shape=(?, 21, 1024, 1), dtype=float32)

layer  6
Tensor("dropout_5/dropout/mul:0", shape=(?, 11, 2048, 1), dtype=float32)

layer  7
Tensor("dropout_6/dropout/mul:0", shape=(?, 6, 4096, 1), dtype=float32)

layer  8
Tensor("dropout_7/dropout/mul:0", shape=(?, 3, 4096, 1), dtype=float32)

layer  9
Tensor("dropout_8/dropout/mul:0", shape=(?, 2, 4096, 1), dtype=float32)

layer  10
Tensor("dropout_9/dropout/mul:0", shape=(?, 1, 4096, 1), dtype=float32)



In [4]:
# reshape the output + Dropout
fl_shape = input_layer.shape
fl_flat = tf.reshape(input_layer, [-1, int(fl_shape[1])* int(fl_shape[2])* int(fl_shape[3])]) #reshape to batch of vectors
do_fl_flat= tf.layers.dropout(fl_flat, rate=0.5, training = True)


In [5]:
############################################
############################### OUTPUT LAYER ####
############################################


logit= tf.layers.dense( inputs= do_fl_flat,
                             units = 10,
                            use_bias = True,
                            kernel_initializer = init_weights,
                            bias_initializer = init_bias)


In [ ]:
y_pred = tf.argmax(logit, axis=-1, output_type=tf.int32) # gives idx of max value in logit
y_real = tf.placeholder(tf.int32, [None,len(y[0])]) # gold standard
y_real_int = tf.argmax(y_real, axis=-1, output_type=tf.int32)

# implement crossentropy with softmax
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_real, logits=logit)
cross_entropy = tf.reduce_mean(cross_entropy)
tf.summary.scalar("cost", cross_entropy)


#### Evaluation (do here for efficency) Kuan says it belongs here ####
correct_prediction = tf.equal(y_pred, y_real_int)
# cast transforms boolean to floats
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Training Step
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

###########################################################
# initilize variables (give them random values)
tf.global_variables_initializer().run()
### tensorboard
loss_summary = tf.summary.scalar("cost", cross_entropy)
train_acc_summary = tf.summary.scalar("train_acc", accuracy)
train_summary = tf.summary.merge((loss_summary, train_acc_summary))
test_summary = tf.summary.scalar("test_acc", accuracy)
writer = tf.summary.FileWriter("/cache/tensorboard-logdir/time_15layer")
writer.add_graph(sess.graph)
############################################################



########## TRAINING PARAMETERS & TRAINING
sample_size = x_train.shape[0]
test_size = x_val.shape[0]
batch_size = 64
steps = 30000
test_step = 100


winner = 0
win_step = 0
for step, idxs in zip(range(steps), pick_batch(sample_size, batch_size)):
    _, stats = sess.run((train_step, train_summary), feed_dict={input_data: x_train[idxs], y_real: y_train[idxs]})
    writer.add_summary(stats, step)
    if not (step % test_step):
        test_acc=[]
        for x_t,y_t in zip(np.split(x_val,6), np.split(y_val,6)): #split test set in 6 parts
            acc = sess.run((accuracy), feed_dict={input_data: x_t, y_real: y_t})
            test_acc.append(acc)
            
        mean_test_acc= np.mean(test_acc)
        print("Step:", step, "Test accuracy: ", mean_test_acc)
        writer.add_summary(sess.run(test_summary, {accuracy: mean_test_acc}), step)
        
        # early stopping
        winner, win_step = early_stop(mean_test_acc, winner, win_step, step, sess)
        if winner == -1:
            break
# clear the writer buffer
writer.flush()
sess.close()

Step: 0 Test accuracy:  0.11333334
Step: 100 Test accuracy:  0.21666665
Step: 200 Test accuracy:  0.30666664
Step: 300 Test accuracy:  0.3033333
Step: 400 Test accuracy:  0.3766667
Step: 500 Test accuracy:  0.38666666
Step: 600 Test accuracy:  0.43000004
Step: 700 Test accuracy:  0.42333332
Step: 800 Test accuracy:  0.39000002
Step: 900 Test accuracy:  0.47333333
Step: 1000 Test accuracy:  0.48666668
Step: 1100 Test accuracy:  0.48666668
Step: 1200 Test accuracy:  0.51
Step: 1300 Test accuracy:  0.5033334
Step: 1400 Test accuracy:  0.51666665
Step: 1500 Test accuracy:  0.5566667
Step: 1600 Test accuracy:  0.54999995
Step: 1700 Test accuracy:  0.52
Step: 1800 Test accuracy:  0.52666664
Step: 1900 Test accuracy:  0.5833333
Step: 2000 Test accuracy:  0.56666666
Step: 2100 Test accuracy:  0.5466666
Step: 2200 Test accuracy:  0.61999995
Step: 2300 Test accuracy:  0.5766667
Step: 2400 Test accuracy:  0.58666664
Step: 2500 Test accuracy:  0.6233333
Step: 2600 Test accuracy:  0.63000005
Step: 